New approach entirely

In [15]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix, classification_report


In [6]:

IMG_SIZE = 224
BATCH_SIZE = 29

# Data paths
TRAIN_DIR = "../Data/train"
VALID_DIR = "../Data/valid"
TEST_DIR  = "../Data/test"



def load_dataset_from_csv(folder_path):
    df = pd.read_csv(os.path.join(folder_path, "labels.csv"))
    image_paths = [os.path.join(folder_path, fname) for fname in df['filename']]
    labels = df['fallen'].values

    def process_image(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    path_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    image_label_ds = path_ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    return image_label_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)



In [7]:
train_ds = load_dataset_from_csv(TRAIN_DIR)
valid_ds = load_dataset_from_csv(VALID_DIR)
test_ds  = load_dataset_from_csv(TEST_DIR)


In [8]:
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary output
])


In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=10)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 350ms/step - accuracy: 0.5844 - loss: 0.8137 - val_accuracy: 0.7966 - val_loss: 0.5074
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 343ms/step - accuracy: 0.7724 - loss: 0.4679 - val_accuracy: 0.7966 - val_loss: 0.6307
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 347ms/step - accuracy: 0.8227 - loss: 0.4532 - val_accuracy: 0.7966 - val_loss: 0.5072
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 346ms/step - accuracy: 0.8079 - loss: 0.4309 - val_accuracy: 0.7966 - val_loss: 0.5407
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 355ms/step - accuracy: 0.7989 - loss: 0.4232 - val_accuracy: 0.7966 - val_loss: 0.5444
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 353ms/step - accuracy: 0.8344 - loss: 0.3740 - val_accuracy: 0.7966 - val_loss: 0.5450
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 361ms/step - accuracy: 0.8305 - loss: 0.3305 - val_accuracy: 0.7966 - val_loss: 0.5835
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 361ms/step - accuracy: 0.8819 - loss: 0.2807 - val_accuracy: 0.

In [10]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.2%}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8888 - loss: 0.2566 
Test Accuracy: 89.83%


In [11]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),  # 👈 input shape goes here
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=20)


Epoch 1/20


/Users/drapcat/4380env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 370ms/step - accuracy: 0.5906 - loss: 0.7405 - val_accuracy: 0.7458 - val_loss: 0.5581
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 369ms/step - accuracy: 0.7387 - loss: 0.5417 - val_accuracy: 0.7966 - val_loss: 1.0029
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 377ms/step - accuracy: 0.7940 - loss: 0.4803 - val_accuracy: 0.8136 - val_loss: 0.5920
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 380ms/step - accuracy: 0.8282 - loss: 0.4551 - val_accuracy: 0.7966 - val_loss: 0.7708
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 385ms/step - accuracy: 0.8495 - loss: 0.4198 - val_accuracy: 0.7966 - val_loss: 0.6510
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 388ms/step - accuracy: 0.8192 - loss: 0.4456 - val_accuracy: 0.8136 - val_loss: 0.7267
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 386ms/step - accuracy: 0.8093 - loss: 0.4366 - val_accuracy: 0.8136 - val_loss: 0.6701
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 389ms/step - accuracy: 0.7673 - loss: 0.4452 - val_accuracy: 0.7966 - val_

In [12]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.2%}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.8888 - loss: 0.3097
Test Accuracy: 89.83%


In [16]:
y_true = []
y_pred = []

for images, labels in test_ds:
    probs = model.predict(images).flatten()
    preds = (probs > 0.5).astype(int)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# Display confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Fallen", "Fallen"]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Confusion Matrix:
[[42  2]
 [ 4 11]]

Classification Report:
              precision    recall  f1-score   support

  Not Fallen       0.91      0.95      0.93        44
      Fallen       0.85      0.73      0.79        15

    accuracy                           0.90        59
   macro avg       0.88      0.84      0.86        59
weighted avg       0.90      0.90      0.90        59

